In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

In [2]:
#load the headlines corpus
corpus_df = pd.read_json("News_Category_Dataset_v2.json", lines=True)
corpus_df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [3]:
sentences = corpus_df["headline"].tolist()
len(sentences)

200853

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F



In [11]:
model = SentenceTransformer('final_model_combination') # I WILL JUST USE THIS MODEL NOW FOR EMBEDDING <-- will be substittued with our best model, \
    #which hopefully be the one with the combination objeective
print(">>>>>>>>>>>>>>>>>>>>>>>>>>> print embeddings for sentences <<<<<<<<<<<<<<<<<< ")

#Our sentences we like to encode
sentences = corpus_df["headline"].tolist()
sentences = sentences[0:40000]

#from hugging faces
#sentence_embeddings= huggingfaces_examples(sentences)
sentence_embeddings = model.encode(sentences) #the shapee of  embeddings is (#sentences, embedding dimension)

print(sentence_embeddings.shape)
print(sentence_embeddings[1].shape)
print(type(sentence_embeddings))
#Print the embeddings
data = []
for sentence, embedding in zip(sentences, sentence_embeddings):
    data.append([sentence,embedding])
    
df=pd.DataFrame(data=data)
df.head()


>>>>>>>>>>>>>>>>>>>>>>>>>>> print embeddings for sentences <<<<<<<<<<<<<<<<<< 
(40000, 768)
(768,)
<class 'numpy.ndarray'>


,0,1
0,There Were 2 Mass Shootings In Texas Last Week...,"[0.30491707, -1.2524736, 0.11239702, 1.1291376..."
1,Will Smith Joins Diplo And Nicky Jam For The 2...,"[0.2988914, 0.25715393, 0.37346962, -0.4477708..."
2,Hugh Grant Marries For The First Time At Age 57,"[-0.4711784, -0.5345203, -0.004865968, 0.49056..."
3,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,"[0.30759478, 0.27576742, -0.3338978, -0.170235..."
4,Julianna Margulies Uses Donald Trump Poop Bags...,"[-0.35530883, 0.82921207, 0.65110743, -0.60549..."


In [12]:
#rename the columns
df = df.rename(columns={0: 'sentence', 1: 'embedding'})
df.head()

,sentence,embedding
0,There Were 2 Mass Shootings In Texas Last Week...,"[0.30491707, -1.2524736, 0.11239702, 1.1291376..."
1,Will Smith Joins Diplo And Nicky Jam For The 2...,"[0.2988914, 0.25715393, 0.37346962, -0.4477708..."
2,Hugh Grant Marries For The First Time At Age 57,"[-0.4711784, -0.5345203, -0.004865968, 0.49056..."
3,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,"[0.30759478, 0.27576742, -0.3338978, -0.170235..."
4,Julianna Margulies Uses Donald Trump Poop Bags...,"[-0.35530883, 0.82921207, 0.65110743, -0.60549..."


In [13]:
searching_word = input("Please enter the word you are searching for: ")
k= int(input("Enter the number of similar sentences you are searching for (k): "))
similarities=[]
word_embedding=model.encode(searching_word)
#word_embedding=huggingfaces_examples(searching_word)

for i in range(sentence_embeddings.shape[0]):
    similarities.append(util.cos_sim(word_embedding, sentence_embeddings[i]).item())
df_temp=df
df_temp["similarity_score"]=similarities
df_temp=df_temp.sort_values(by=['similarity_score'],ascending=False)
df_temp.head()
print("The word you searched for is {}".format(searching_word))
print("The {} sentences similar to your input are".format(k))
print(df_temp.iloc[0:k]['sentence'].to_string(index=False))
    


The word you searched for is dog
The 3 sentences similar to your input are
Elizabeth Warren: Donald Trump's Presidency 'Fe...
Dogs At Polling Stations Are Getting The UK Thr...
Texas Church Shooter Claimed He Used Dogs As Ta...


In [15]:
searching_word = input("Please enter the word you are searching for: ")
k= int(input("Enter the number of similar sentences you are searching for (k): "))
similarities=[]
word_embedding=model.encode(searching_word)
#word_embedding=huggingfaces_examples(searching_word)

for i in range(sentence_embeddings.shape[0]):
    similarities.append(util.cos_sim(word_embedding, sentence_embeddings[i]).item())
df_temp=df
df_temp["similarity_score"]=similarities
df_temp=df_temp.sort_values(by=['similarity_score'],ascending=False)
df_temp.head()
print("The word you searched for is {}".format(searching_word))
print("The {} sentences similar to your input are".format(k))
print(df_temp.iloc[0:k]['sentence'].to_string(index=False))
    


The word you searched for is queen
The 3 sentences similar to your input are
    This Obit Is A Worthy Tribute To Princess Leia
'Princess Complicit' Ivanka Trump Skewered Over...
What Female Leaders Can Learn From The Game Of ...
